In [1]:
from qiskit import *
from qiskit.tools.visualization import plot_histogram
from qiskit.tools.monitor import job_monitor
from qiskit_ibm_provider import IBMProvider
import math

In [2]:
def create_circuit(num_bits):
    circuit = QuantumCircuit(num_bits, num_bits)
    circuit.h(range(num_bits)) 
    return circuit

In [3]:
def string_to_blocks(strng):
    blocks = []
    for char in strng:
        blocks.append((ord(char)))
    return blocks

In [4]:
def block_xor(key, block):
    return key ^ block

def block_cipher(key, plaintext_blocks):
    return [block_xor(key, block) for block in plaintext_blocks]


In [5]:
def oracle(circuit, known_plaintext_blocks, ciphertext_blocks, num_key_qubits):
    for block_index, known_plaintext in enumerate(known_plaintext_blocks):
        ciphertext = ciphertext_blocks[block_index]

        for i in range(num_key_qubits):
            if (ciphertext >> i) & 1 == (known_plaintext >> i) & 1:
                circuit.x(i)
        
        circuit.h(num_key_qubits - 1)
        circuit.mcx(list(range(num_key_qubits - 1)), num_key_qubits - 1)
        circuit.h(num_key_qubits - 1) 

        for i in range(num_key_qubits):
            if (ciphertext >> i) & 1 == (known_plaintext >> i) & 1:
                circuit.x(i)

    return circuit


In [6]:
def diffuser(circuit, num_qubits):
    circuit.h(range(num_qubits))
    circuit.x(range(num_qubits))

    circuit.h(num_qubits - 1)
    circuit.mcx(list(range(num_qubits - 1)), num_qubits - 1)
    circuit.h(num_qubits - 1)

    circuit.x(range(num_qubits))
    circuit.h(range(num_qubits))

    return circuit


In [ ]:
def main():
    key = 0b1011110110
    num_bits = 10
    known_plaintext = "Lockheed has field propulsion craft"
    
    plaintext_blocks = string_to_blocks(known_plaintext)
    ciphertext_blocks = block_cipher(key, plaintext_blocks)

    circuit = create_circuit(num_bits)
    num_iterations = int(math.sqrt(2**num_bits))
    
    for _ in range(num_iterations):
        circuit = oracle(circuit, plaintext_blocks, ciphertext_blocks, num_bits)
        circuit = diffuser(circuit, num_bits)
    circuit.measure(range(num_bits), range(num_bits))

    simulator = Aer.get_backend('qasm_simulator') 
    result = execute(circuit, simulator, shots=1024).result()
    
#     provider = IBMProvider()
#     provider.active_account()
#     qcomp = provider.get_backend('ibm_kyoto')
#     job = execute(circuit, backend = qcomp)
#     job_monitor(job)
#     result = job.result()
    
    counts = result.get_counts(circuit)
    sorted_counts = dict(sorted(counts.items(), key=lambda item: item[1], reverse=True))
    top_counts = dict(list(sorted_counts.items())[:5])
    
    probable_key = max(counts, key=counts.get)
    probable_key = int(probable_key, 2)
    decrypted_blocks = block_cipher(probable_key, ciphertext_blocks) # Decrypt using key for verification
    
    display(plot_histogram(top_counts))
    print(f"Most Probable Key: {format(probable_key, f'0{num_bits}b')}")
    print(plaintext_blocks == decrypted_blocks)

In [ ]:
if __name__ == "__main__":
    main()